In [30]:
import os
import numpy as np
from scipy.fftpack import fft
from scipy.io import wavfile
from scipy import signal
from glob import glob
import re
import pandas as pd
import gc
from scipy.io import wavfile

from keras.models import load_model
from keras.utils.np_utils import to_categorical
from keras import optimizers, losses, activations, models
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import keras

In [50]:
new_sample_rate = 8000

LABELS = ['_silence', '_unknown', 'down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'up', 'yes']
TRAIN_PATH = './input/train/audio/'
TEST_PATH = './input/test/audio/'
OUTPUT_PATH = './output/'
MODEL_NAME = 'cnn_aug_300_no_cw_2.h5'

class_weight = {0: 13.0,
 1: 1.0,
 2: 9.0,
 3: 9.0,
 4: 9.0,
 5: 9.0,
 6: 9.0,
 7: 9.0,
 8: 9.0,
 9: 9.0,
 10: 9.0,
 11: 9.0}

In [32]:
# 원본 클래스 비율
# {0: 1200,
#  1: 41039,
#  2: 2359,
#  3: 2372,
#  4: 2353,
#  5: 2375,
#  6: 2357,
#  7: 2367,
#  8: 2367,
#  9: 2380,
#  10: 2375,
#  11: 2377}

In [33]:
## custom_fft and log_specgram functions written by DavidS.
def custom_fft(y, fs):
    T = 1.0 / fs
    N = y.shape[0]
    yf = fft(y)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    # FFT 는 대칭(simmetrical)이므로 반쪽만 얻음.
    # FFT 는 복소수이므로 실수값만 취하기 위해 abs()
    vals = 2.0/N * np.abs(yf[0:N//2])
    return xf, vals

def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [34]:
## utility function to grab all wav files inside train data folder.
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [35]:
def pad_audio(samples, L=16000):
    '''
    pad audios that are less than 16000(1 second) with 0s to make them all have the same length.
    '''
    if len(samples) >= L: 
        return samples
    else: 
        return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0)) 
        # sample 앞뒤로 constant_values[0]과 constant_values[1]을 각각 pad_width 갯수 만큼 패딩
        # 총길이는 len(samples) + 2*pad_width

# def chop_audio(samples, L=16000, num=200):
#     '''
#     chop audios that are larger than 16000(eg. wav files in background noises folder) to 16000 in length.
#     create several chunks out of one large wav files given the parameter 'num'.
#     '''
#     for i in range(num):
#         beg = np.random.randint(0, len(samples) - L)
#         yield samples[beg: beg + L]

def label_transform(labels):
    '''
    레이블 정규화 및 one-hot벡터화 (더미화)
    '''
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('_silence')
        elif label not in LABELS:
            nlabels.append('_unknown')
        else:
            nlabels.append(label)
    encoder = LabelEncoder()
    encoder.fit(nlabels)
    nlabels = encoder.transform(nlabels)
    return nlabels

# 1. Load Data & Preprocessing

### 1) Load labels, fnames

In [36]:
labels, fnames = list_wavs_fname(TRAIN_PATH)

./input/train/audio/


### 2) Feature Extraction

In [37]:
# sample_rate, samples = wavfile.read(os.path.join(TRAIN_PATH, labels[0], fnames[0]))
# len(samples)

In [38]:
%%time
y = []
X = []
for i, (label, fname) in enumerate(zip(labels, fnames)):
    sample_rate, samples = wavfile.read(os.path.join(TRAIN_PATH, label, fname))
    if len(samples) > 16000:
        pass
    else:
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        y.append(label)
        X.append(specgram)

CPU times: user 1min 31s, sys: 2.54 s, total: 1min 33s
Wall time: 1min 32s


/home/chi/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [39]:
X = np.array(X)
X = X.reshape(tuple(list(X.shape) + [1])) # (64841, 99, 81, 1) 로 reshape
y = to_categorical(label_transform(y))

### 3) Train Validation Set Split

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, random_state=1130) # 9:1로 train, valid 셋 나눔.
del X, y
gc.collect()

0

In [41]:
# model = load_model(os.path.join(OUTPUT_PATH, MODEL_NAME))

# 2. Modeling

In [26]:
input_shape = (99, 81, 1) # in order to fit into Conv2D layer, we need to reshape it.
nclass = 12

In [ ]:
## Modeling
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu, padding='same')(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu, padding='same')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=2, activation=activations.relu, padding='same')(img_1)
img_1 = Convolution2D(16, kernel_size=2, activation=activations.relu, padding='same')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu, padding='same')(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu, padding='same')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu, padding='same')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(img_1)
dense_1 = Dense(512, activation=activations.relu)(dense_1)
dense_1 = Dropout(rate=0.2)(dense_1)
dense_1 = Dense(512, activation=activations.relu)(dense_1)
dense_1 = Dropout(rate=0.2)(dense_1)
dense_1 = Dense(512, activation=activations.relu)(dense_1)
dense_1 = Dropout(rate=0.2)(dense_1)
dense_1 = Dense(128, activation=activations.relu)(dense_1)
dense_1 = Dropout(rate=0.2)(dense_1)
dense_1 = Dense(64, activation=activations.relu)(dense_1)
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam(lr=0.001)

model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 99, 81, 1)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 99, 81, 1)         4         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 99, 81, 8)         40        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 99, 81, 8)         264       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 40, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 49, 40, 8)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 49, 40, 16)        528       
__________

In [51]:
# 네트워크 시각화
# plot_model(model, to_file='output/model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
%%time
# model.fit(X_train, y_train, batch_size=2048, validation_data=(X_valid, y_valid), \
#           class_weight=class_weight, epochs=300, shuffle=True, verbose=1)
model.fit(X_train, y_train, batch_size=2048, validation_data=(X_valid, y_valid), \
          epochs=300, shuffle=True, verbose=1)
model.save(os.path.join(OUTPUT_PATH, MODEL_NAME))

Train on 93216 samples, validate on 4907 samples
Epoch 1/300
93216/93216 [==============================] - 28s - loss: 1.6157 - acc: 0.6159 - val_loss: 2.3386 - val_acc: 0.6309
Epoch 2/300
93216/93216 [==============================] - 27s - loss: 1.3985 - acc: 0.6262 - val_loss: 2.1232 - val_acc: 0.6328
Epoch 3/300
93216/93216 [==============================] - 27s - loss: 1.0761 - acc: 0.6557 - val_loss: 1.9151 - val_acc: 0.7312
Epoch 4/300
93216/93216 [==============================] - 27s - loss: 0.7871 - acc: 0.7378 - val_loss: 1.6688 - val_acc: 0.7976
Epoch 5/300
93216/93216 [==============================] - 27s - loss: 0.6236 - acc: 0.7934 - val_loss: 1.4626 - val_acc: 0.8237
Epoch 6/300
93216/93216 [==============================] - 27s - loss: 0.5288 - acc: 0.8270 - val_loss: 1.2106 - val_acc: 0.8592
Epoch 7/300
93216/93216 [==============================] - 27s - loss: 0.4650 - acc: 0.8487 - val_loss: 0.9987 - val_acc: 0.8433
Epoch 8/300
93216/93216 [=======================

In [ ]:
model.fit(X_train, y_train, batch_size=2048, validation_data=(X_valid, y_valid), \
          class_weight=class_weight, epochs=300, shuffle=True, verbose=1)
model.save(os.path.join(OUTPUT_PATH, MODEL_NAME))

Train on 93216 samples, validate on 4907 samples
Epoch 1/300
93216/93216 [==============================] - 27s - loss: 0.5189 - acc: 0.9623 - val_loss: 0.1099 - val_acc: 0.9719
Epoch 2/300
93216/93216 [==============================] - 26s - loss: 0.5298 - acc: 0.9568 - val_loss: 0.0927 - val_acc: 0.9733
Epoch 3/300
93216/93216 [==============================] - 26s - loss: 0.4385 - acc: 0.9633 - val_loss: 0.0993 - val_acc: 0.9690
Epoch 4/300
93216/93216 [==============================] - 26s - loss: 0.3931 - acc: 0.9664 - val_loss: 0.0910 - val_acc: 0.9753
Epoch 5/300
93216/93216 [==============================] - 26s - loss: 0.3670 - acc: 0.9670 - val_loss: 0.1141 - val_acc: 0.9660
Epoch 6/300
93216/93216 [==============================] - 26s - loss: 0.3437 - acc: 0.9692 - val_loss: 0.1004 - val_acc: 0.9711
Epoch 7/300
93216/93216 [==============================] - 26s - loss: 0.3217 - acc: 0.9699 - val_loss: 0.0905 - val_acc: 0.9737
Epoch 8/300
93216/93216 [=======================

93216/93216 [==============================] - 26s - loss: 0.1912 - acc: 0.9805 - val_loss: 0.0792 - val_acc: 0.9766
Epoch 88/300
93216/93216 [==============================] - 26s - loss: 0.1749 - acc: 0.9804 - val_loss: 0.0868 - val_acc: 0.9739
Epoch 89/300
93216/93216 [==============================] - 26s - loss: 0.1738 - acc: 0.9816 - val_loss: 0.0834 - val_acc: 0.9792
Epoch 90/300
93216/93216 [==============================] - 26s - loss: 0.1837 - acc: 0.9816 - val_loss: 0.0775 - val_acc: 0.9792
Epoch 91/300
93216/93216 [==============================] - 26s - loss: 0.1761 - acc: 0.9820 - val_loss: 0.0793 - val_acc: 0.9782
Epoch 92/300
93216/93216 [==============================] - 26s - loss: 0.1744 - acc: 0.9824 - val_loss: 0.0825 - val_acc: 0.9772
Epoch 93/300
93216/93216 [==============================] - 26s - loss: 0.1691 - acc: 0.9821 - val_loss: 0.0816 - val_acc: 0.9788
Epoch 94/300
93216/93216 [==============================] - 26s - loss: 0.1802 - acc: 0.9809 - val_loss

93216/93216 [==============================] - 26s - loss: 0.1493 - acc: 0.9830 - val_loss: 0.0868 - val_acc: 0.9778
Epoch 150/300
93216/93216 [==============================] - 26s - loss: 0.1601 - acc: 0.9832 - val_loss: 0.0860 - val_acc: 0.9755
Epoch 151/300
93216/93216 [==============================] - 26s - loss: 0.1552 - acc: 0.9831 - val_loss: 0.0852 - val_acc: 0.9772
Epoch 152/300
93216/93216 [==============================] - 26s - loss: 0.1703 - acc: 0.9824 - val_loss: 0.0809 - val_acc: 0.9790
Epoch 153/300
93216/93216 [==============================] - 26s - loss: 0.1689 - acc: 0.9830 - val_loss: 0.0865 - val_acc: 0.9766
Epoch 154/300
93216/93216 [==============================] - 26s - loss: 0.1600 - acc: 0.9838 - val_loss: 0.0911 - val_acc: 0.9766
Epoch 155/300
93216/93216 [==============================] - 26s - loss: 0.1650 - acc: 0.9824 - val_loss: 0.0936 - val_acc: 0.9749
Epoch 156/300
93216/93216 [==============================] - 26s - loss: 0.1537 - acc: 0.9832 - v

# 3. Validation

In [42]:
preds_proba = model.predict(X_valid, batch_size=1024, verbose=1)

4907/4907 [==============================] - 0s     


In [43]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.98288

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         3      3075     1    4     0    3    1    2      2     2   
down             0         2   170    1     0    0    0    0      0     0   
go               0         4     0  158     0    3    0    0      0     2   
left             0         6     0    0   171    0    0    0      0     0   
no               0         1     1    3     0  172    0    0      0     0   
off              1         2     0    0     0    0  162    2      0     0   
on               0         6     0    1     0    0    4  155      0     0   
right            2         2     0    0     1    0    0    0    153     0   
stop             1         3     0    0     0    0    0    0      0   198   
up               0         2 

In [12]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.97493

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         5      3014     1    7     4    6    6    9     20    12   
down             0         2   169    2     0    1    0    0      0     0   
go               0         5     0  160     0    1    0    0      0     0   
left             0         1     0    0   173    0    0    2      0     1   
no               0         0     0    2     0  175    0    0      0     1   
off              1         0     0    1     0    0  166    1      0     0   
on               0         2     1    2     0    0    1  160      0     0   
right            2         2     0    0     0    0    0    0    154     0   
stop             0         0     0    0     0    0    1    1      0   200   
up               0         0 

In [14]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.97330

 -------------------------- 

preds     _silence  _unknown  down   go  left   no  off   on  right  stop  \
actuals                                                                     
_silence        63         0     0    0     0    0    0    0      0     0   
_unknown         9      3013     4    7     5    5    7   13     13     8   
down             0         1   171    1     0    1    0    0      0     0   
go               0         3     0  158     0    2    0    0      1     2   
left             0         1     0    0   172    1    1    2      0     0   
no               0         0     1    1     0  175    0    0      0     0   
off              1         1     0    1     0    0  166    1      0     0   
on               0         4     0    1     0    0    1  159      0     0   
right            3         1     0    0     0    0    0    0    154     0   
stop             1         1     0    0     0    0    1    1      0   197   
up               1         0 

In [26]:
preds = [LABELS[i] for i in np.argmax(preds_proba, axis=1)]
actuals = [LABELS[i] for i in np.argmax(y_valid, axis=1)]
print('* 정확도 : %.5f' % (np.sum(np.array(actuals) == np.array(preds)) / float(len(actuals))))
preds = pd.Categorical(preds, categories=LABELS)
actuals = pd.Categorical(actuals, categories=LABELS)
print('\n -------------------------- \n')
print(pd.crosstab(actuals, preds, rownames=['actuals'], colnames=['preds']))
print('\n -------------------------- \n')
print(classification_report(actuals, preds))

* 정확도 : 0.97028

 -------------------------- 

preds     _silence  _unknown  down  go  left   no  off  on  right  stop   up  \
actuals                                                                        
_silence        54         0     0   0     0    0    0   0      0     0    0   
_unknown         4      2029     0   7     1    9    2   1      0     0    4   
down             0         4   128   1     0    1    0   0      0     0    0   
go               0         5     4  87     0    2    0   0      0     0    0   
left             0         6     0   0   126    0    0   0      0     0    2   
no               0         4     2   0     0  103    0   0      0     1    0   
off              1         6     1   0     1    0  100   0      0     1    1   
on               0         4     0   0     0    0    1  98      0     0    0   
right            0         5     0   0     1    0    0   0    121     0    0   
stop             0         4     1   0     0    1    0   0      0   118  

# 제출파일 생성

In [44]:
%%time
X = []
submission_fpaths = sorted(glob(os.path.join(TEST_PATH, r'*wav')))
for fpath in submission_fpaths:
    sample_rate, samples = wavfile.read(fpath)
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: 
        n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        X.append(specgram)
        
X = np.array(X)
X = X.reshape(tuple(list(X.shape) + [1]))

CPU times: user 2min 26s, sys: 3.8 s, total: 2min 30s
Wall time: 2min 27s


In [45]:
preds_proba = model.predict(X, batch_size=2048, verbose=1)
preds = [[L.replace('_', '') for L in LABELS][i] for i in np.argmax(preds_proba, axis=1)]

158538/158538 [==============================] - 14s    


In [46]:
df = pd.DataFrame({'fname': submission_fpaths, 'label': preds})
df['fname'] = df['fname'].apply(lambda p: p.split('/')[-1])
df.to_csv(os.path.join(OUTPUT_PATH, 'sub_' + MODEL_NAME.split('.')[0] + '.csv'), index=False)

In [47]:
df['label'].value_counts()

unknown    100591
yes          6519
off          6272
stop         6064
left         5637
no           5428
up           5392
go           5237
down         5135
on           5126
right        4923
silence      2214
Name: label, dtype: int64

In [16]:
df['label'].value_counts()

unknown    81043
on         14810
off         7550
up          7044
right       6950
left        6659
go          6553
stop        6360
yes         6292
no          6199
down        4760
silence     4318
Name: label, dtype: int64

In [18]:
df['label'].value_counts()

unknown    82108
on         15084
off         8341
no          6978
right       6541
left        6510
up          6284
yes         6137
stop        5775
go          5678
down        4916
silence     4186
Name: label, dtype: int64